In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

In [2]:
!nvidia-smi

Fri May  6 17:48:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.utils as utils
import gzip
!pip install wandb
import os
import wandb
! pip install torchsummaryX
from torchsummaryX import summary
!pip install pytorch_warmup
import pytorch_warmup as warmup
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

checkpoints_store_path = "/content/drive/MyDrive/02710_proj/store_checkpoints"
create_folder(checkpoints_store_path)

batch_size = 1
epochs = 20
lr = 1e-3

model_id = f"WSCNN"
data_path = checkpoints_store_path + model_id
create_folder(data_path)
instance_length = 40
instance_stride = 5
# wandb.init(project="02710", entity="saltedfish",name=model_id)
# wandb.config = {
#   "learning_rate": lr,
#   "epochs": epochs,
#   "batch_size": batch_size,
# }


     |████████████████████████████████| 1.8 MB 654 kB/s 
     |████████████████████████████████| 181 kB 56.9 MB/s 
     |████████████████████████████████| 144 kB 65.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=032f279115247758d59a568e4ea41d673f234d1888569fc27fd2ca65c3369068
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
     |████████████████████████████████| 314 kB 652 kB/s 
  Created wheel for pytorch-warmup: filename=pytorch_warmup-0.1.0-py3-none-any.whl size=5802 sha256=31c35a53cc70007f48a65b936d656a0267abbc98dbbfede4d23404bf0856309b
  Stored in directory: /root/.cache/pip/wheels/c7/5d/24/9475e442daa1e9332c122c79fb5131b9e4e91946009365902e
Successfully built pytorch-warmup


In [4]:
a_file = gzip.open("/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/training_sample_0/sequences.fa.gz", "rb")
contents = a_file.readlines()

for i in range(10):
    print(contents[i])

b'> chr1,+,866159,866259; class:0\n'
b'GCCTCGGTCCTGTTCTTCCTGATGCGTGTCTGCTGAGGCCAGGAGCTGGCTTTGGCCCAT\n'
b'GGGCCTGTCCTAGTGGGAGGCCCCAGCATGTTGAGCCAGTA\n'
b'> chr1,+,870491,870591; class:0\n'
b'GGATTGGGCTGAATTAGCAAGAAGAGGAGAAATGAGGGAAGAAAAGAGTTAAATGCATGT\n'
b'TGATTCCAAGCCCCCGCCTGCCGGGGGGACAGCGGGAGGTT\n'
b'> chr1,+,949147,949247; class:0\n'
b'AGGTAAAGGGAGGCCACGGGATGGCGGTGGGCAGCTGGCCTTCTAGTAACGAGCCCTCAG\n'
b'TGCCTTCTGTGCCTGGGGTCCCTGCCGGCGGGATGTAGAGG\n'
b'> chr1,+,949360,949460; class:0\n'


# Sample preparation

In [5]:
def read_file_helper(file_path):
    res = []
    with gzip.open(file_path, "rb") as f:
        for line in f.readlines():
            line = str(line)[2:-3]
            
            if line.startswith(">"):
                indicator = 0
                tmp = []
                loc, y = line.strip().split(";")
                chr_num, sign, start, end = loc[2:].split(",")
                tmp.extend([chr_num, sign, int(start), int(end), y[-1]])
            
            else:
                indicator += 1
                tmp.append(line)
                if indicator == 2:
                    res.append(tmp)
    df = pd.DataFrame(res, columns = ["chr_num", "sign", "start", "end", "y", "seq_part1", "seq_part2"])
    df["seq"] = df["seq_part1"] + df["seq_part2"]
    df["y"] = df["y"].astype(int)
    return df
                

def embed(sequence, instance_len, instance_stride):
    instance_num = int((len(sequence) - instance_len) / instance_stride) + 1
    bag = []
    for i in range(instance_num):
        instance = sequence[i * instance_stride:i * instance_stride + instance_len]
        instance = one_hot_encode(instance)
        bag.append(instance)
    bag = np.stack(bag).astype(float)
    return bag

def one_hot_encode(seq):
    arrays = [np.array([1, 0, 0, 0]),
             np.array([0, 1, 0, 0]),
             np.array([0, 0, 1, 0]),
             np.array([0, 0, 0, 1]),
             np.array([0.25, 0.25, 0.25, 0.25])]
             
    mapping = dict(zip("ACGTN", arrays))
   
    return np.vstack([mapping[i] for i in seq])

def create_bag(seqs, instance_len=instance_length, instance_stride=instance_stride):
    bags = []
    for seq in seqs:
        # rev_seq = reverse_complement(seq) 
        # seq = seq + "N"*instance_length + rev_seq
        bags.append(embed(seq, instance_len, instance_stride)) 
        
    return np.array(bags).astype(float)


class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path):
        df = read_file_helper(data_path)
        df["seq"] = df["seq_part1"] + df["seq_part2"]
        # print(df["seq"])
        # df["rev_seq"] = df["seq"].apply(lambda seq : reverse_complement(seq))
        # print(df["rev_seq"])

        # df["seq"] = df["seq"] + "N"*instance_length + df["rev_seq"]
        self.X, self.Y = create_bag(df["seq"]), df["y"].to_numpy()
        
        assert len(self.X) == len(self.Y)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, item):
        y = [0, 1] if self.Y[item] == 1 else [1, 0]
        return self.X[item], np.array(y)

In [6]:
def reverse_complement(seq):
    mapping = str.maketrans("ATCG","TAGC")
    return seq.translate(mapping)[::-1]

# seq = "AAAAAATTTNNNTCCCC"
# rev_seq = reverse_complement("AAAAAATTTNNNTCCCC")
# seq = seq + "N"*instance_length + rev_seq
# print(seq)

In [7]:
train_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/training_sample_0/sequences.fa.gz"
valid_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/test_sample_0/sequences.fa.gz"

train_data = LibriSamples(train_data_path)
valid_data = LibriSamples(valid_data_path)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size, shuffle=True)

# y_num_0 = []
# y_num_1 = []
for x, y in train_loader:
    print(x, y)
    break
    # if y == 0:
    #   y_num_0.append(y)
    # else:
    #   y_num_1.append(y)

tensor([[[[1., 0., 0., 0.],
          [0., 0., 0., 1.],
          [0., 1., 0., 0.],
          ...,
          [0., 0., 1., 0.],
          [0., 1., 0., 0.],
          [0., 1., 0., 0.]],

         [[0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [0., 1., 0., 0.],
          ...,
          [0., 0., 0., 1.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.]],

         [[0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 0., 1.],
          ...,
          [0., 0., 0., 1.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.]],

         ...,

         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [1., 0., 0., 0.],
          ...,
          [0., 1., 0., 0.],
          [1., 0., 0., 0.],
          [1., 0., 0., 0.]],

         [[1., 0., 0., 0.],
          [0., 0., 0., 1.],
          [0., 1., 0., 0.],
          ...,
          [0., 0., 1., 0.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.]],

         [[0., 0., 1., 0.],
          [0., 0., 1., 0.],
    

In [8]:
print(x.shape)
# print(len(y_num_0))
# print(len(y_num_1))

torch.Size([1, 13, 40, 4])


# WeakRM

In [ ]:
class WeakRM(nn.Module):
    """
    used for channel = 4, AGCT
    """

    def __init__(self, training=True):
        super().__init__()

        self.inst_conv1 = nn.Sequential(
            nn.Conv1d(4, 32, kernel_size=15, padding=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )

        self.inst_conv2 = nn.Sequential(
            nn.Conv1d(32, 16, kernel_size=5, padding=2, stride=1),
            nn.ReLU()
        )

        self.dropout = nn.Dropout(0.2)

        self.attention_v = nn.Sequential(
            nn.Linear(320, 128),
            nn.Tanh()
        )

        self.attention_u = nn.Sequential(
            nn.Linear(320, 128),
            nn.Sigmoid()
        )

        self.attention_weights = nn.Sequential(
            nn.Linear(128, 1),
        )

        self.cls = nn.Sequential(
            nn.Linear(320, 2),
            nn.Sigmoid()
        )
        self.softmax = nn.Softmax()

    def forward(self, inputs, training=True, mask=None):
        inputs = torch.squeeze(inputs, 0)
        inputs = inputs.permute((0, 2, 1))  # torch.Size([13, 5, 40])
        inst_conv1 = self.inst_conv1(inputs)  # torch.Size([13, 32, 20])
        inst_conv1 = self.dropout(inst_conv1)
        inst_conv2 = self.inst_conv2(inst_conv1)

        inst_features = nn.Flatten()(inst_conv2)

        attention_v = self.attention_v(inst_features)
        attention_u = self.attention_v(inst_features)

        # print(attention_u*attention_v)
        # print(self.attention_weights(attention_u * attention_v))

        gated_attention = self.attention_weights(attention_u * attention_v).permute((1, 0))
        # print(gated_attention)

        gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])

        bag_features = torch.matmul(gated_attention, inst_features)

        bag_probability = self.cls(bag_features)

        return bag_probability, gated_attention

In [ ]:
x = torch.rand((1, 13, 40, 4))
encoder = WeakRM()
summary(encoder, x)

                              Kernel Shape  Output Shape   Params Mult-Adds
Layer                                                                      
0_inst_conv1.Conv1d_0          [4, 32, 15]  [13, 32, 40]   1.952k     76.8k
1_inst_conv1.ReLU_1                      -  [13, 32, 40]        -         -
2_inst_conv1.MaxPool1d_2                 -  [13, 32, 20]        -         -
3_dropout                                -  [13, 32, 20]        -         -
4_inst_conv2.Conv1d_0          [32, 16, 5]  [13, 16, 20]   2.576k     51.2k
5_inst_conv2.ReLU_1                      -  [13, 16, 20]        -         -
6_attention_v.Linear_0          [320, 128]     [13, 128]  41.088k    40.96k
7_attention_v.Tanh_1                     -     [13, 128]        -         -
8_attention_v.Linear_0          [320, 128]     [13, 128]        -    40.96k
9_attention_v.Tanh_1                     -     [13, 128]        -         -
10_attention_weights.Linear_0     [128, 1]       [13, 1]    129.0     128.0
11_softmax  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_inst_conv1.Conv1d_0,"[4, 32, 15]","[13, 32, 40]",1952.0,76800.0
1_inst_conv1.ReLU_1,-,"[13, 32, 40]",NaN,NaN
2_inst_conv1.MaxPool1d_2,-,"[13, 32, 20]",NaN,NaN
3_dropout,-,"[13, 32, 20]",NaN,NaN
4_inst_conv2.Conv1d_0,"[32, 16, 5]","[13, 16, 20]",2576.0,51200.0
5_inst_conv2.ReLU_1,-,"[13, 16, 20]",NaN,NaN
6_attention_v.Linear_0,"[320, 128]","[13, 128]",41088.0,40960.0
7_attention_v.Tanh_1,-,"[13, 128]",NaN,NaN
8_attention_v.Linear_0,"[320, 128]","[13, 128]",NaN,40960.0


In [10]:
class WSCNN(nn.Module):

    def __init__(self, instance_len=40, merging='MAX', training=True):
        super(WSCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 16, kernel_size=(1, 15), padding=(0, 7)),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(1, 1)),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 1, kernel_size=(1, 1)),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(0.5)
        self.pool1 = nn.MaxPool2d((instance_len, 1))

        if merging == 'MAX':
            self.pool2 = nn.AdaptiveMaxPool2d((1, 1))
            # self.pool2 = nn.GlobalMaxPooling2d()
        elif merging == 'AVG':
            self.pool2 = nn.AdaptiveAvgPool2d((1, 1))
            # self.pool2 = nn.GlobalAveragePooling2d()

        self.cls = nn.Sequential(
            nn.Linear(1, 2),
            nn.Sigmoid()
        )

    def forward(self, inputs, training=True, mask=None):
        inputs = inputs.permute((0, 3, 2, 1))
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.dropout(x)
        x = self.conv3(x)
        out = self.pool2(x)
        out = self.cls(out)
        return out[0][0],_

x = torch.rand((1, 13, 40, 4))
encoder = WSCNN()
summary(encoder, x)

                    Kernel Shape     Output Shape Params Mult-Adds
Layer                                                             
0_conv1.Conv2d_0  [4, 16, 1, 15]  [1, 16, 40, 13]  976.0    499.2k
1_conv1.ReLU_1                 -  [1, 16, 40, 13]      -         -
2_pool1                        -   [1, 16, 1, 13]      -         -
3_conv2.Conv2d_0  [16, 32, 1, 1]   [1, 32, 1, 13]  544.0    6.656k
4_conv2.ReLU_1                 -   [1, 32, 1, 13]      -         -
5_dropout                      -   [1, 32, 1, 13]      -         -
6_conv3.Conv2d_0   [32, 1, 1, 1]    [1, 1, 1, 13]   33.0     416.0
7_conv3.ReLU_1                 -    [1, 1, 1, 13]      -         -
8_pool2                        -     [1, 1, 1, 1]      -         -
9_cls.Linear_0            [1, 2]     [1, 1, 1, 2]    4.0       2.0
10_cls.Sigmoid_1               -     [1, 1, 1, 2]      -         -
--------------------------------------------------------------------
                        Totals
Total params            1.557

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1.Conv2d_0,"[4, 16, 1, 15]","[1, 16, 40, 13]",976.0,499200.0
1_conv1.ReLU_1,-,"[1, 16, 40, 13]",NaN,NaN
2_pool1,-,"[1, 16, 1, 13]",NaN,NaN
3_conv2.Conv2d_0,"[16, 32, 1, 1]","[1, 32, 1, 13]",544.0,6656.0
4_conv2.ReLU_1,-,"[1, 32, 1, 13]",NaN,NaN
5_dropout,-,"[1, 32, 1, 13]",NaN,NaN
6_conv3.Conv2d_0,"[32, 1, 1, 1]","[1, 1, 1, 13]",33.0,416.0
7_conv3.ReLU_1,-,"[1, 1, 1, 13]",NaN,NaN
8_pool2,-,"[1, 1, 1, 1]",NaN,NaN


In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score, average_precision_score

# model = WeakRM().cuda()
model = WSCNN().cuda()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)
criterion = nn.BCELoss(weight=torch.tensor([0.8, 0.2])).cuda()
num_steps = len(train_loader) * epochs
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)
warmup_scheduler = warmup.RAdamWarmup(optimizer)


for epoch in range(1, epochs + 1):
    # batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    # training
    model.train()
    num_correct = 0
    total_loss = 0
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.float().cuda()
        y = y.float().cuda()

        outputs,_ = model(x)
        
        # print(outputs[0].shape)
        # print(y[0].shape)
        loss = criterion(outputs[0], y[0])

        outputs = torch.argmax(outputs)

        num_correct += int((outputs == torch.argmax(y)).sum())
        total_loss += loss

        train_acc = 100 * num_correct / ((i + 1) * batch_size)
        train_loss = float(total_loss / (i + 1))
    
        loss.backward()
        optimizer.step()
        with warmup_scheduler.dampening():
            scheduler.step()
            
#         if i // 100 == 0:
#             batch_bar.set_postfix(
#                 acc="{:.04f}%".format(train_acc),
#                 loss="{:.04f}".format(train_loss),
#                 num_correct=num_correct,
#                 lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
#             batch_bar.update()
    
#     batch_bar.close()

    print_content ="Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch,
        epochs,
        100 * num_correct / (len(train_loader) * batch_size),
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])
    )

    print(print_content)

    # eval
    model.eval()
    
    num_correct = 0
    total_loss = 0
    predictions =[]
    label = []
    for i, (x, y) in enumerate(valid_loader):
        x = x.float().cuda()
        y = y.float().cuda()

        with torch.no_grad():
            outputs_probs,_ = model(x)

        outputs = torch.argmax(outputs_probs)

        num_correct += int((outputs == torch.argmax(y)).sum())
        total_loss += loss
        
        predictions.append(outputs_probs.detach().cpu().numpy()[0])
        label.append(torch.argmax(y).detach().cpu().numpy())
    # print(_)    
    dev_acc = 100 * num_correct / len(valid_loader)
    dev_loss = total_loss / len(valid_loader)
        
    auROC = roc_auc_score(np.array(label).flatten(), np.array(predictions)[:, 1])
    print("Eval Acc {:.04f}%, Eval auROC {:.04f}%".format(dev_acc, auROC))
    
#     torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, path)



Epoch 1/20: Train Acc 76.9533%, Train Loss 0.2579, Learning Rate 0.0010
Eval Acc 80.0000%, Eval auROC 0.8389%
Epoch 2/20: Train Acc 81.8500%, Train Loss 0.1893, Learning Rate 0.0010
Eval Acc 82.7500%, Eval auROC 0.8465%
Epoch 3/20: Train Acc 82.8200%, Train Loss 0.1860, Learning Rate 0.0009
Eval Acc 83.1700%, Eval auROC 0.8458%
Epoch 4/20: Train Acc 82.5233%, Train Loss 0.1860, Learning Rate 0.0009
Eval Acc 84.0300%, Eval auROC 0.8478%
Epoch 5/20: Train Acc 82.7367%, Train Loss 0.1854, Learning Rate 0.0009
Eval Acc 83.8200%, Eval auROC 0.8434%
Epoch 6/20: Train Acc 82.6700%, Train Loss 0.1853, Learning Rate 0.0008
Eval Acc 84.0000%, Eval auROC 0.8454%
Epoch 7/20: Train Acc 82.8000%, Train Loss 0.1851, Learning Rate 0.0007
Eval Acc 83.0700%, Eval auROC 0.8451%
Epoch 8/20: Train Acc 82.6133%, Train Loss 0.1846, Learning Rate 0.0007
Eval Acc 83.9300%, Eval auROC 0.8455%
Epoch 9/20: Train Acc 82.8967%, Train Loss 0.1834, Learning Rate 0.0006
Eval Acc 83.7600%, Eval auROC 0.8484%
Epoch 10/2

In [ ]:
# from tqdm import tqdm
# from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
# from sklearn.metrics import precision_score, recall_score
# from sklearn.metrics import roc_auc_score, average_precision_score

# model = WeakRM().cuda()
# optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.005)
# # criterion = nn.BCELoss()
# criterion = nn.BCELoss(weight=torch.tensor([0.8, 0.2])).cuda()
# num_steps = len(train_loader) * epochs
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)
# # warmup_scheduler = warmup.RAdamWarmup(optimizer)


# for epoch in range(1, epochs + 1):
#     batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

#     num_correct = 0
#     total_loss = 0

#     # training samples
#     for i, (x, y) in enumerate(train_loader):
#         optimizer.zero_grad()

#         x = x.float().cuda()
#         y = y.float().cuda()

#         outputs,_ = model(x)

#         # print(outputs[0].shape)
#         # print(y.shape)

#         loss = criterion(outputs[0], y[0])

#         outputs = torch.argmax(outputs)

#         # outputs = torch.tensor([0 if outputs[0][0]<0.5 else 1]).cuda()

#         num_correct += int((outputs == y).sum())
#         total_loss += float(loss)

#         train_acc = 100 * num_correct / ((i + 1) * batch_size)
#         train_loss = float(total_loss / (i + 1))
        
#         # tqdm lets you add some details so you can monitor training as you train.
#         batch_bar.set_postfix(
#             acc="{:.04f}%".format(train_acc),
#             loss="{:.04f}".format(train_loss),
#             num_correct=num_correct,
#             lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

#         # Another couple things you need for FP16.
#         loss.backward()  # This is a replacement for loss.backward()
#         optimizer.step()  # This is a replacement for optimizer.step()
#         # with warmup_scheduler.dampening():
#         #     scheduler.step()
    
#         batch_bar.update()  # Update tqdm bar
#     print(_)
    
#     batch_bar.close()  # You need this to close the tqdm bar

#     print_content ="Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f} \n".format(
#     epoch,
#     epochs,
#     100 * num_correct / (len(train_loader) * batch_size),
#     float(total_loss / len(train_loader)),
#     float(optimizer.param_groups[0]['lr']))

#     print(print_content)

#     batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Eval')

#     num_correct = 0
#     total_loss = 0

#     # eval samples
#     model.eval()
#     predictions =[]
#     label = []
#     for i, (x, y) in enumerate(valid_loader):
#         x = x.float().cuda()
#         y = y.float().cuda()

#         with torch.no_grad():
#             outputs,_ = model(x)

#         # outputs = torch.tensor([0 if outputs[0][0]<0.5 else 1]).cuda()
#         # print(outputs)

#         outputs = torch.argmax(outputs)
#         num_correct += int((outputs == y).sum())
#         total_loss += float(loss)
        
#         predictions.append(outputs.detach().cpu().numpy())
#         label.append(y.detach().cpu().numpy())


#         dev_acc = 100 * num_correct / ((i + 1) * batch_size)
#         dev_loss = float(total_loss / (i + 1))
#         # print(dev_loss)

#         # tqdm lets you add some details so you can monitor training as you train.
#         batch_bar.set_postfix(
#             acc="{:.04f}%".format(dev_acc),
#             loss="{:.04f}".format(dev_loss),
#             num_correct=num_correct,
#             lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

#         # Another couple things you need for FP16.
#         batch_bar.update()  # Update tqdm bar

#     batch_bar.close()  # You need this to close the tqdm bar


#     auROC = roc_auc_score(y_true=label, y_score=predictions)

#     # wandb.log({"Train Acc:":train_acc, "Train loss:": train_loss,"Test Acc ":dev_acc,"Test loss":dev_loss,"auROC":auROC,"lr":lr})
#     print(auROC)
#     print_content ="Epoch {}/{}: Eval Acc {:.04f}%, Eval auROC {:.04f}%\n".format(
#     epoch,
#     epochs,
#     dev_acc,
#     auROC)
#     print(print_content)
    
#     path = data_path+f"/model_epoch_{epoch}.ckpt"
#     torch.save({'epoch': epoch,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, path)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.1220, 0.1194, 0.0968, 0.0888, 0.0895, 0.0618, 0.0523, 0.0650, 0.0534,
         0.0582, 0.0610, 0.0676, 0.0642]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
Epoch 1/20: Train Acc 100.0000%, Train Loss 0.1773, Learning Rate 0.0010 



ValueError: ignored

# Tensorflow version

## Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Input

tfk = tf.keras
tfkl = tf.keras.layers
tfkc = tf.keras.callbacks


class Noisyand(tf.keras.layers.Layer):

    def __init__(self, a=7.5, **kwargs):
        super(Noisyand, self).__init__(**kwargs)
        self.a = a

    def build(self, input_shape):
        self.b = self.add_weight(shape=(input_shape[-1],),
                                 initializer=tfk.initializers.RandomUniform(minval=0, maxval=1),
                                 constraint=tfk.constraints.MinMaxNorm(min_value=0.0, max_value=1.0),
                                 trainable=True)

    def call(self, inputs, training=True, mask=None):
        if len(inputs.shape) == 4:
            inputs = tf.squeeze(inputs, 2)
        part1 = sigmoid((tf.reduce_mean(inputs, axis=1) - self.b) * self.a) - sigmoid(-self.a * self.b)
        part2 = sigmoid(self.a * (1 - self.b)) - sigmoid(-self.a * self.b)
        return part1 / part2

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


class WSCNN(tf.keras.Model):

    def __init__(self, instance_len=40, merging='MAX', a=7.5):
        super(WSCNN, self).__init__()

        assert merging in ['MAX', 'AVG', 'NOISY']

        self.conv1 = tfkl.Conv2D(16, (1, 15), padding='same',
                                 activation='relu', kernel_regularizer=l2(0.005))
        self.conv2 = tfkl.Conv2D(32, (1, 1), padding='same',
                                 activation='relu', kernel_regularizer=l2(0.005))
        self.conv3 = tfkl.Conv2D(1, (1, 1), padding='same',
                                 activation='sigmoid', kernel_regularizer=l2(0.005))
        self.dropout = tfkl.Dropout(0.5)
        self.pool1 = tfkl.MaxPool2D(pool_size=(1, instance_len))

        if merging == 'MAX':
            self.pool2 = tfkl.GlobalMaxPooling2D()
        elif merging == 'AVG':
            self.pool2 = tfkl.GlobalAveragePooling2D()
        elif merging == 'NOISY':
            self.pool2 = Noisyand(a=a)

    def call(self, inputs, training=True, mask=None):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.conv3(x)
        out = self.pool2(x)
        return out

    def model(self):
        x = Input(shape=(13, 40, 5))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))


class WSCNNLSTM(tf.keras.Model):

    def __init__(self, merging='MAX', a=7.5):
        super(WSCNNLSTM, self).__init__()

        assert merging in ['MAX', 'AVG', 'NOISY']  ## 如果merging不是在这几个值之间，那么就会报错

        # 2D, batch = 1, 确保每一个输入的instance都是共享参数的CNN提取feature
        self.conv1 = tf.keras.layers.Conv2D(16, (1, 15), padding='same', activation='relu',
                                            kernel_regularizer=l2(0.005))
        self.conv2 = tf.keras.layers.Conv1D(1, 1, padding='same', activation='sigmoid',
                                            kernel_regularizer=l2(0.005))
        self.lstm = tf.keras.layers.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
        self.dropout1 = tfkl.Dropout(0.2)
        self.dropout2 = tfkl.Dropout(0.5)
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(1, 5))
        if merging == 'MAX':
            self.pool2 = tfkl.GlobalMaxPooling1D()
        elif merging == 'AVG':
            self.pool2 = tfkl.GlobalAveragePooling1D()
        elif merging == 'NOISY':
            self.pool2 = Noisyand(a=a)

    def call(self, inputs, training=True, mask=None):
        x = self.conv1(inputs)
        x = self.pool1(x)
        if training:
            x = self.dropout1(x, training=training)
        x = self.lstm(x)
        if training:
            x = self.dropout2(x, training=training)
        x = self.conv2(x)
        out = self.pool2(x)

        return out

    def model(self):
        x = Input(shape=(13, 40, 4))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))


class WeakRM(tf.keras.Model):

    def __init__(self):
        super(WeakRM, self).__init__()
        # 1D batch = 6, 保证每一条instance的参数都是由CNN来提取的
        self.conv1 = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        self.conv2 = tfkl.Conv1D(16, 5, padding='same', activation='relu',
                                 kernel_regularizer=l2(0.005))
        self.dropout = tfkl.Dropout(0.2)
        self.pool1 = tfkl.MaxPool1D(pool_size=2)

        self.att_v = tfkl.Dense(128, activation='tanh')
        self.att_u = tfkl.Dense(128, activation='sigmoid')

        self.attention_weights = tfkl.Dense(1)

        self.classifier = tfkl.Dense(1, activation='sigmoid')

    def call(self, inputs, training=True, mask=None):
        #   (1,none, instance_length, 4)
        input_bag = tf.squeeze(inputs, axis=0)
        ## squeeze function is to remove all the shape equal 1

        ## so we could use the convolutional
        inst_conv1 = self.conv1(input_bag)
        inst_pool1 = self.pool1(inst_conv1)
        inst_pool1 = self.dropout(inst_pool1, training=training)

        inst_conv2 = self.conv2(inst_pool1)

        inst_features = tfkl.Flatten()(inst_conv2)

        attention_vmatrix = self.att_v(inst_features)
        attention_umatrix = self.att_u(inst_features)

        gated_attention = self.attention_weights(attention_vmatrix * attention_umatrix)
        print(gated_attention.shape)

        gated_attention = tf.transpose(gated_attention, perm=[1, 0])
        gated_attention = tfkl.Softmax()(gated_attention)

        bag_features = tf.matmul(gated_attention, inst_features)

        bag_probability = self.classifier(bag_features)

        return bag_probability, gated_attention

    # def model(self):
    #     x = Input(shape=(13, 40, 5))
    #     return tf.keras.Model(inputs=[x], outputs=self.call(x))


class WeakRMLSTM(tf.keras.Model):

    def __init__(self):
        super(WeakRMLSTM, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(16, (1, 15), padding='same', activation='relu')
        # self.conv1 = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        # self.conv2 = tfkl.Conv1D(16, 5, padding='same', activation='relu',
        #                          kernel_regularizer=l2(0.005))
        self.lstm = tf.keras.layers.TimeDistributed(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)))
        self.dropout = tfkl.Dropout(0.2)
        self.pool1 = tfkl.MaxPool2D(pool_size=(1, 2))

        self.att_v = tfkl.Dense(128, activation='tanh')
        self.att_u = tfkl.Dense(128, activation='sigmoid')

        self.attention_weights = tfkl.Dense(1)

        self.classifier = tfkl.Dense(1, activation='sigmoid')

    def call(self, inputs, training=True, mask=None):
        # input_bag = tf.squeeze(inputs, axis=0)
        input_bag = inputs
        inst_conv1 = self.conv1(input_bag)
        inst_pool1 = self.pool1(inst_conv1)
        if training:
            inst_pool1 = self.dropout(inst_pool1, training=training)

        inst_conv2 = self.lstm(inst_pool1)

        inst_features = tf.squeeze(inst_conv2, axis=0)

        attention_vmatrix = self.att_v(inst_features)
        attention_umatrix = self.att_u(inst_features)

        gated_attention = self.attention_weights(attention_vmatrix * attention_umatrix)

        gated_attention = tf.transpose(gated_attention, perm=[1, 0])
        gated_attention = tfkl.Softmax()(gated_attention)

        # tf.stop_gradient(gated_attention)

        bag_features = tf.matmul(gated_attention, inst_features)

        bag_probability = self.classifier(bag_features)

        return bag_probability, gated_attention

    def model(self):
        x = Input(shape=(13, 40, 5))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))


"""

======================================================================================================
ADDING THE STRUCTURE INFORMATION INTO THE MODEL

"""


class WeakRM_struc(tf.keras.Model):

    def __init__(self):
        super(WeakRM_struc, self).__init__()
        # 1D batch = 6, 保证每一条instance的参数都是由CNN来提取的
        self.conv1 = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        self.conv2 = tfkl.Conv1D(16, 5, padding='same', activation='relu',
                                 kernel_regularizer=l2(0.005))

        self.conv1_str = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        self.conv2_str = tfkl.Conv1D(16, 5, padding='same', activation='relu',
                                     kernel_regularizer=l2(0.005))

        self.dropout = tfkl.Dropout(0.2)
        self.pool1 = tfkl.MaxPool1D(pool_size=2)

        self.att_v = tfkl.Dense(128, activation='tanh')
        self.att_u = tfkl.Dense(128, activation='sigmoid')

        self.attention_weights = tfkl.Dense(1)

        self.classifier = tfkl.Dense(1, activation='sigmoid')

    def call(self, train_seq, train_str, training=True, mask=None):
        ###############################################
        ### sequence sequence
        ###############################################
        #
        input_bag_seq = tf.squeeze(train_seq, axis=0)
        inst_conv1_seq = self.conv1(input_bag_seq)
        inst_pool1_seq = self.pool1(inst_conv1_seq)
        inst_pool1_seq = self.dropout(inst_pool1_seq, training=training)

        inst_conv2_seq = self.conv2(inst_pool1_seq)
        inst_features_seq = tfkl.Flatten()(inst_conv2_seq)

        ###############################################
        ### structure sequence
        ###############################################

        input_bag_str = tf.squeeze(train_str, axis=0)
        inst_conv1_str = self.conv1_str(input_bag_str)
        inst_pool1_str = self.pool1(inst_conv1_str)
        inst_pool1_str = self.dropout(inst_pool1_str, training=training)

        inst_conv2_str = self.conv2_str(inst_pool1_str)
        inst_features_str = tfkl.Flatten()(inst_conv2_str)
        # inst_features = tfkl.Flatten()(inst_conv2_str)

        ###############################################
        ### combine two type sequence
        ###############################################

        inst_features = tf.concat([inst_features_seq, inst_features_str], 0)

        attention_vmatrix = self.att_v(inst_features)
        attention_umatrix = self.att_u(inst_features)

        gated_attention = self.attention_weights(attention_vmatrix * attention_umatrix)

        gated_attention = tf.transpose(gated_attention, perm=[1, 0])
        gated_attention = tfkl.Softmax()(gated_attention)

        bag_features = tf.matmul(gated_attention, inst_features)

        bag_probability = self.classifier(bag_features)

        return bag_probability, gated_attention


class WeakRM_struc_only(tf.keras.Model):

    def __init__(self):
        super(WeakRM_struc_only, self).__init__()
        self.conv1 = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        self.conv2 = tfkl.Conv1D(16, 5, padding='same', activation='relu',
                                 kernel_regularizer=l2(0.005))

        self.conv1_str = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        self.conv2_str = tfkl.Conv1D(16, 5, padding='same', activation='relu',
                                     kernel_regularizer=l2(0.005))

        self.lstm = tf.keras.layers.LSTM(16)

        self.dropout = tfkl.Dropout(0.2)
        self.pool1 = tfkl.MaxPool1D(pool_size=2)

        self.att_v = tfkl.Dense(128, activation='tanh')
        self.att_u = tfkl.Dense(128, activation='sigmoid')

        self.attention_weights = tfkl.Dense(1)

        self.classifier = tfkl.Dense(1, activation='sigmoid')

    def call(self, train_seq, train_str, training=True, mask=None):
        ###############################################
        ### sequence sequence
        ###############################################
        #
        input_bag_seq = tf.squeeze(train_seq, axis=0)
        inst_conv1_seq = self.conv1(input_bag_seq)
        inst_pool1_seq = self.pool1(inst_conv1_seq)
        inst_pool1_seq = self.dropout(inst_pool1_seq, training=training)

        inst_conv2_seq = self.conv2(inst_pool1_seq)
        inst_features_seq = tfkl.Flatten()(inst_conv2_seq)

        attention_vmatrix_seq = self.att_v(inst_features_seq)
        attention_umatrix_seq = self.att_u(inst_features_seq)

        gated_attention_seq = self.attention_weights(attention_vmatrix_seq * attention_umatrix_seq)

        gated_attention_seq = tf.transpose(gated_attention_seq, perm=[1, 0])
        gated_attention_seq = tfkl.Softmax()(gated_attention_seq)

        bag_features_seq = tf.matmul(gated_attention_seq, inst_features_seq)

        ###############################################
        ### structure sequence
        ###############################################

        # input_bag_str = tf.squeeze(train_str, axis=0)
        inst_conv1_str = self.conv1_str(train_str)
        inst_pool1_str = self.pool1(inst_conv1_str)
        inst_pool1_str = self.dropout(inst_pool1_str, training=training)

        inst_conv2_str = self.lstm(inst_pool1_str)
        # inst_features_str = tfkl.Flatten()(inst_conv2_str)
        # inst_conv2_str = tfkl.Flatten()(inst_pool1_str)

        ###############################################
        ### combine two type sequence
        ###############################################

        bag_features_cbd = tf.concat([bag_features_seq, inst_conv2_str], 1)

        # bag_probability = self.classifier(bag_features_cbd)
        bag_probability = self.classifier(bag_features_cbd)

        # return bag_probability, gated_attention_seq
        return bag_probability, inst_conv2_str


# model = WeakRM()
# # model = WeakRMLSTM()
# # model = WSCNN()

# # model.build(input_shape=(1, None, 40, 4))
# # print(model.summary())
# print(model.model().summary())

## TF data preparation

In [ ]:
train_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/training_sample_0/sequences.fa.gz"
valid_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/test_sample_0/sequences.fa.gz"

train_data = read_file_helper(train_data_path)
train_data["seq"] = train_data["seq_part1"] + train_data["seq_part2"]
train_data, train_label = create_bag(train_data["seq"]), train_data["y"].to_numpy().reshape(-1,1)

# print(train_data)
# print(train_label)

In [ ]:
valid_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/test_sample_0/sequences.fa.gz"
valid_data = read_file_helper(valid_data_path)
valid_data["seq"] = valid_data["seq_part1"] + valid_data["seq_part2"]
valid_data, valid_label = create_bag(valid_data["seq"]), valid_data["y"].to_numpy().reshape(-1,1)


In [ ]:
# print(valid_data)
print(valid_data.shape)
print(train_data.shape)

(10000, 13, 40, 4)
(30000, 13, 40, 4)


In [ ]:
print(type(train_label[0]))
train_label.shape

<class 'numpy.ndarray'>


(30000, 1)

In [ ]:
len(valid_data)
x = []
for i in range(len(valid_data)):
  x.append(valid_data[i])
x = np.asarray(x)

In [ ]:
import argparse
import itertools
import os
import time

instance_len =40
train_dataset = tf.data.Dataset.from_generator(lambda: itertools.zip_longest(train_data, train_label),
                                               output_types=(tf.float32, tf.int32),
                                               output_shapes=(tf.TensorShape([None, instance_len, 4]),
                                                              tf.TensorShape([None])))
valid_dataset = tf.data.Dataset.from_generator(lambda: itertools.zip_longest(valid_data, valid_label),
                                               output_types=(tf.float32, tf.int32),
                                               output_shapes=(tf.TensorShape([None, instance_len, 4]),
                                                              tf.TensorShape([None])))
train_dataset = train_dataset.shuffle(100).batch(1)
valid_dataset = valid_dataset.batch(1)



In [ ]:
for x, y in train_dataset:
  print(x,y)
  break

In [ ]:
import tensorflow as tf
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Input

tfk = tf.keras
tfkl = tf.keras.layers
tfkc = tf.keras.callbacks
class WeakRM(tf.keras.Model):

    def __init__(self):
        super(WeakRM, self).__init__()
        # 1D batch = 6, 保证每一条instance的参数都是由CNN来提取的
        self.conv1 = tfkl.Conv1D(32, 15, padding='same', activation='relu')
        self.conv2 = tfkl.Conv1D(16, 5, padding='same', activation='relu',
                                 kernel_regularizer=l2(0.005))
        self.dropout = tfkl.Dropout(0.2)
        self.pool1 = tfkl.MaxPool1D(pool_size=2)

        self.att_v = tfkl.Dense(128, activation='tanh')
        self.att_u = tfkl.Dense(128, activation='sigmoid')

        self.attention_weights = tfkl.Dense(1)

        self.classifier = tfkl.Dense(1, activation='sigmoid')

    def call(self, inputs, training=True, mask=None):
        #   (1,none, instance_length, 4)
        input_bag = tf.squeeze(inputs, axis=0)
        ## squeeze function is to remove all the shape equal 1

        ## so we could use the convolutional
        inst_conv1 = self.conv1(input_bag)
        inst_pool1 = self.pool1(inst_conv1)
        inst_pool1 = self.dropout(inst_pool1, training=training)

        inst_conv2 = self.conv2(inst_pool1)

        inst_features = tfkl.Flatten()(inst_conv2)

        attention_vmatrix = self.att_v(inst_features)
        attention_umatrix = self.att_u(inst_features)

        gated_attention = self.attention_weights(attention_vmatrix * attention_umatrix)
        print(gated_attention)

        gated_attention = tf.transpose(gated_attention, perm=[1, 0])
        gated_attention = tfkl.Softmax()(gated_attention)
        # print(np.array(gated_attention))
        # print(np.sum(gated_attention))

        bag_features = tf.matmul(gated_attention, inst_features)

        bag_probability = self.classifier(bag_features)

        return bag_probability, gated_attention
    def model(self):
        x = Input(shape=(13, 40, 5))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

class WSCNN(tf.keras.Model):

    def __init__(self, instance_len=40, merging='MAX', a=7.5):
        super(WSCNN, self).__init__()

        assert merging in ['MAX', 'AVG', 'NOISY']

        self.conv1 = tfkl.Conv2D(16, (1, 15), padding='same',
                                 activation='relu', kernel_regularizer=l2(0.005))
        self.conv2 = tfkl.Conv2D(32, (1, 1), padding='same',
                                 activation='relu', kernel_regularizer=l2(0.005))
        self.conv3 = tfkl.Conv2D(1, (1, 1), padding='same',
                                 activation='sigmoid', kernel_regularizer=l2(0.005))
        self.dropout = tfkl.Dropout(0.5)
        self.pool1 = tfkl.MaxPool2D(pool_size=(1, instance_len))

        if merging == 'MAX':
            self.pool2 = tfkl.GlobalMaxPooling2D()
        elif merging == 'AVG':
            self.pool2 = tfkl.GlobalAveragePooling2D()
        elif merging == 'NOISY':
            self.pool2 = Noisyand(a=a)

    def call(self, inputs, training=True, mask=None):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.conv3(x)
        out = self.pool2(x)
        return out

    def model(self):
        x = Input(shape=(13, 40, 5))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

# model = WeakRM()
model = WSCNN()
print(model.model().summary())



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 13, 40, 5)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 40, 16)        1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 1, 16)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 1, 32)         544       
                                                                 
 dropout_1 (Dropout)         (None, 13, 1, 32)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 1, 1)          33        
                                                           

In [ ]:
import sys 
# set the optimization
opt = tf.keras.optimizers.Adam(lr=5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-5)

train_loss = tf.keras.metrics.Mean()
valid_loss = tf.keras.metrics.Mean()
train_auROC = tf.keras.metrics.AUC()
valid_auROC = tf.keras.metrics.AUC()

## Specify the input size

# model = WeakRM()
model = WSCNN()

instance_len =40

train_step_signature = [
    tf.TensorSpec(shape=(1, None, instance_len, 4), dtype=tf.float32),
    tf.TensorSpec(shape=(1, 1), dtype=tf.int32)
]

# @tf.function(input_signature=train_step_signature)
def train_step(train_seq, train_label):
    with tf.GradientTape() as tape:
        output_probs, _ = model(train_seq, training=True)
        # print("Train Gated Attenion:",_)
        # print(_.shape)
        # attention = _.numpy()
        # print(attention)
        # tf.print(_ ,output_stream=sys.stdout)

        loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)(y_true=train_label, y_pred=output_probs)
        total_loss = loss + tf.reduce_sum(model.losses)  ## calculate the total_loss of each tensor (including the l2 & l1 regularization)
        gradients = tape.gradient(total_loss, model.trainable_variables)

        ## use the optimizor to update the gradient
        opt.apply_gradients(
            zip(gradients, model.trainable_variables))  ## based on the gradient to upgrade the model weights
        train_loss(loss)  ## loss is calculated from binary crossentropy
        train_auROC(y_true=train_label, y_pred=output_probs)



# @tf.function(input_signature=train_step_signature)
def valid_step(valid_seq, valid_label):
    inf_probs, _ = model(valid_seq, training=False)
    # print("Val Gated Attention:")
    # tf.print(_ ,output_stream=sys.stdout)
    vloss = tf.keras.losses.BinaryCrossentropy()(y_true=valid_label, y_pred=inf_probs)
    valid_loss(vloss)
    valid_auROC(y_true=valid_label, y_pred=inf_probs)


EPOCHS = 100
current_monitor = np.inf
patient_count = 0


for epoch in tf.range(1, EPOCHS + 1):
    train_loss.reset_states()
    valid_loss.reset_states()

    train_auROC.reset_states()
    valid_auROC.reset_states()

    epoch_start_time = time.time()
    for tdata in train_dataset:
        train_step(tdata[0], tdata[1])  ## train the dataset,tdata[0] stands for sequences (bags), tdata[1]:label
        sys.exit()
    print('Training of epoch {} finished! Time cost is {}s'.format(epoch, round(time.time() - epoch_start_time, 2)))

    valid_start_time = time.time()
    for vdata in valid_dataset:
        valid_step(vdata[0], vdata[1])  ## train the validation dataset

    new_valid_monitor = np.round(valid_loss.result().numpy(), 4)
    if new_valid_monitor < current_monitor:
        print('val_loss improved from {} to {}'.
              format(str(current_monitor), str(new_valid_monitor)))
        current_monitor = new_valid_monitor
        patient_count = 0
    else:
        print('val_loss did not improved from {}'.format(str(current_monitor)))
        patient_count += 1

    if patient_count == 5:
        break

    template = "Epoch {}, Time Cost: {}s, TL: {}, TROC: {}, VL:{}, VROC: {}"
    print(template.format(epoch, str(round(time.time() - valid_start_time, 2)),
                          str(np.round(train_loss.result().numpy(), 4)),
                          str(np.round(train_auROC.result().numpy(), 4)),
                          str(np.round(valid_loss.result().numpy(), 4)),
                          str(np.round(valid_auROC.result().numpy(), 4)),
                          )
          )


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


ValueError: ignored

# WeakRM

In [ ]:
# !git clone https://github.com/daiyun02211/WeakRM.git /content/drive/MyDrive/weakRM_Demo
%cd /content/drive/MyDrive/weakRM_Demo

/content/drive/MyDrive/weakRM_Demo


In [ ]:
!python ./Scripts/token2npy.py --input_dir='./Data/m7G/' --output_dir='./Data/m7G/processed/'

Traceback (most recent call last):
  File "./Scripts/token2npy.py", line 3, in <module>
    from utils import create_folder, embed
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 818, in get_code
  File "<frozen importlib._bootstrap_external>", line 917, in get_data
KeyboardInterrupt


In [ ]:
!python ./Scripts/main.py --training=True --input_dir='./Data/m7G/processed/' --model_name=WSCNN --fusion_method=MAX

Loading data ...
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
2022-05-05 21:53:37.664362: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Creating model ...
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
Training of epoch 1 finished! Time cost is 35.13s
val_loss improved from inf to 0.7123, saving closed
Epoch 1, Time Cost: 1.45s, TL: 0.7058, TROC: 0.5255, VL:0.7123, VROC: 0.6392
Training of epoch 2 finished! Time cost is 16.81s
val_loss improved from 0.7123 to 0.6799, saving closed
Epoch 2, Time Cost: 1.07s, TL: 0.6823, TROC: 0.5874, VL:0.6799, VROC: 0.6478
Training of epoch 3 finished! Time cost is 16.56s
val_loss improved from 0.6799 to